# Задача 

Напишите агента, набирающего как можно больше баллов в игре MountainCar-V0

# Методы решения 

Попробуем решить задачу двумя способами и сравнить результаты:
- Сделаем выборку из большого числа "успешных" игр и обучим нейронную сеть
    - Так как мы получаем награду, отличную от $-1$ только в том случае, когда машинка пересекает отметку в $0.5$ (что при случайных действиях случается экстремально редко), изменим функцию награды так, что значение награды за итерацию увеличивается на $1$, если мы достигли отметку в $-0.18$ (это значение нам подойдёт, так как стартовая позиция принимает значения от $-0.6$ до $-0.4$).
    - Игра будет считаться успешной, если за $200$ максимально возможных итераций суммарное значение награды составило больше $-199$.
- DQN
    - Мы не будем просто собирать данные и обучать нейронную сеть на их основе. Вместо этого мы будем создавать тренировочные данные прямо после каждой попытки и будем обучать сеть сразу же после попытки.

# Решение

In [170]:
import gym
import random
import numpy as np
import tensorflow as tf 
from collections import deque
from tensorflow.keras.models     import Sequential
from tensorflow.keras.layers     import Dense, Dropout
from tensorflow.keras.optimizers import Adam

Сначала напишем метод, который будет сравнивать эффективность моделей. Он будет принимать модель, которая играет $100$ игр и считает среднее количество награды за игру (расчёт награды не меняется относительно условия задачи).

In [145]:
def model_test(test_model):
    
    # Список из суммарных наград за 1 игру.
    scores = [] 
    
    games_number = 100
    
    # Пробегаемся по 100 играм.
    for each_game in range(games_number):    
        
        score = 0
        
        # Состояние среды за предыдущую итерацию. 
        # Будем его передавать в качестве параметра для нашей модели.
        previous_observation = []
        
        for step_index in range(goal_steps):
            
            # Если это первая итерация, то выбераем случайное действие.
            if len(previous_observation)==0:
                action = random.randrange(0,2)
            # Иначе подаём модели прошлой состояние и выбираем действие с наибольшим весом.
            else:
                action = np.argmax(test_model.predict(previous_observation.reshape(-1, env.observation_space.shape[0]))[0])

            new_observation, reward, done, info = env.step(action)
            previous_observation = new_observation
            score += reward
            
            if done:            
                break
                
        env.reset()
        scores.append(score)
            
    print('Средняя награда за {} игр: {}'.format(games_number, sum(scores)/len(scores)))

## Решение №1 

Напишем класс, содержащий всю логику решения.

In [228]:
class NN:
    def __init__(self, env):
        self.env = env
        
        self.goal_steps = 200
        self.score_requirement = -199
        self.games_number = 100000

        self.model = self.create_model()

    # Создание модели.
    def create_model(self):
        model = Sequential([
            Dense(512, input_dim=self.env.observation_space.shape[0], activation='relu'),
            Dense(256, input_dim=input_size, activation='relu'),
            Dense(128, input_dim=input_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.env.action_space.n, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(), metrics=['acc'])
        return model
    
    # Обучение модели.
    def train_model(self, x, y):        
        self.model.fit(x, y, epochs=5)

    # Подготовка данных для обучения.
    def data_preparation(self):
        
        # Сюда будем записывать информацию об играх, которые прошли отбор по награде.
        training_data = []
        
        # Сюда будем записывать количество очков в играх, которые прошли отбор.
        accepted_scores = []
        
        # Пробегаемся по всем играм.
        for game_index in range(self.games_number):
            
            score = 0
            
            # Сюда запишем всю историю
            game_memory = []
            previous_observation = []
            
            for step_index in range(self.goal_steps):
                action = random.randrange(0, 3)
                observation, reward, done, info = env.step(action)

                if len(previous_observation) > 0:
                    game_memory.append([previous_observation, action])

                previous_observation = observation
                if observation[0] > -0.18:
                    reward = 1

                score += reward
                if done:
                    break

            # Если к концу игры мы набрали нужное количество награды, то 
            # все состояния и принятые решение записываем в training data.
            if score >= self.score_requirement:
                accepted_scores.append(score)
                for data in game_memory:
                    if data[1] == 1:
                        output = [0, 1, 0]
                    elif data[1] == 0:
                        output = [1, 0, 0]
                    elif data[1] == 2:
                        output = [0, 0, 1]
                    training_data.append([data[0], output])

            env.reset()
            
            if (game_index%1000==0):
                print('Завершено {} игр из {}.'.format(game_index, self.games_number))

        print("Количество игр, данные по которым будут использованы для обучения: ",len(accepted_scores))
        
        # Составляем массивы для обучения (x – состояние, y – решение)
        x = np.array([i[0] for i in training_data]).reshape(-1, self.env.observation_space.shape[0])
        y = np.array([i[1] for i in training_data]).reshape(-1, self.env.action_space.n)

        return x, y
    def save_model(self, filepath):
        self.model.save(filepath)

Протестируем данный метод.

In [223]:
env = gym.make("MountainCar-v0")
env.reset()

array([-0.43422332,  0.        ])

In [224]:
nn_agent = NN(env)

In [225]:
x, y = nn_agent.data_preparation()

Завершено 0 игр из 100000.
Завершено 1000 игр из 100000.
Завершено 2000 игр из 100000.
Завершено 3000 игр из 100000.
Завершено 4000 игр из 100000.
Завершено 5000 игр из 100000.
Завершено 6000 игр из 100000.
Завершено 7000 игр из 100000.
Завершено 8000 игр из 100000.
Завершено 9000 игр из 100000.
Завершено 10000 игр из 100000.
Завершено 11000 игр из 100000.
Завершено 12000 игр из 100000.
Завершено 13000 игр из 100000.
Завершено 14000 игр из 100000.
Завершено 15000 игр из 100000.
Завершено 16000 игр из 100000.
Завершено 17000 игр из 100000.
Завершено 18000 игр из 100000.
Завершено 19000 игр из 100000.
Завершено 20000 игр из 100000.
Завершено 21000 игр из 100000.
Завершено 22000 игр из 100000.
Завершено 23000 игр из 100000.
Завершено 24000 игр из 100000.
Завершено 25000 игр из 100000.
Завершено 26000 игр из 100000.
Завершено 27000 игр из 100000.
Завершено 28000 игр из 100000.
Завершено 29000 игр из 100000.
Завершено 30000 игр из 100000.
Завершено 31000 игр из 100000.
Завершено 32000 игр и

In [226]:
nn_agent.train_model(x, y)

Epoch 1/5
46168/46168 [==============================] - 7s 157us/sample - loss: 0.2213 - acc: 0.3787
Epoch 2/5
46168/46168 [==============================] - 8s 163us/sample - loss: 0.2193 - acc: 0.4005
Epoch 3/5
46168/46168 [==============================] - 7s 149us/sample - loss: 0.2188 - acc: 0.3998
Epoch 4/5
46168/46168 [==============================] - 7s 147us/sample - loss: 0.2184 - acc: 0.4027
Epoch 5/5
46168/46168 [==============================] - 7s 146us/sample - loss: 0.2184 - acc: 0.4021


In [232]:
model_test(nn_agent.model)

Средняя награда за 100 игр: -137.27


In [233]:
nn_agent.save_model("NNmodel.h5")

## Решение №2

Напишем класс, содержащий всю логику решения.

In [ ]:
class DQN:
    def __init__(self, env):
        self.env     = env
        
        # Сюда будем складывать информацию о всех играх.
        self.memory  = deque(maxlen=2000)
        
        # Параметр, на который будет снижаться награда с последующими итерациями.
        self.gamma = 0.85
        
        # Разброс коэффицента, в зависимости от которого мы будем совершать случайное
        # действие в конкретном случае. Вначале он большой, так как у нас нет
        # никакой информации о среде.
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        
        # Коэффицент, на который мы будем снижать эпсилон после каждой успешной итерации.
        self.epsilon_decay = 0.995
        
        self.learning_rate = 0.005
        
        # Коэффицент для пересчёта весов model в веса target_model.
        self.tau = .125

        # model – непосредственно для предсказания шага, мы делаем.
        # target_model – для предсказания шага, который мы хотим сделать.
        self.model        = self.create_model()
        self.target_model = self.create_model()

    # Создание модели.
    def create_model(self):
        model = Sequential([
            Dense(24, input_dim=self.env.observation_space.shape[0], activation="relu"),
            Dense(48, activation="relu"),
            Dense(24, activation="relu"),
            Dense(self.env.action_space.n)
        ])
        model.compile(loss="mean_squared_error", optimizer=Adam(lr=self.learning_rate), metrics=['acc'])
        return model

    # Метод, который выдаёт действие, которое мы в конечном итоге применяем.
    # Будет ли действие случайным или же оно будет выдоно моделью зависит от
    # текущих параметров эпсилон.
    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    # Добавление итераций в память.
    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    # Обучаем сеть на основе предыдущих итераций.
    def replay(self):
        batch_size = 32
        
        # Если у нас ещё недостаточно данных, ничего не делоем.
        if len(self.memory) < batch_size: 
            return
        
        # Берём случайных срез данных.
        samples = random.sample(self.memory, batch_size)
        
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            
            # Если мы дошли до последней итерации или дошли до значения 0.5, то
            # мы больше не получим последующих наград.
            if done:
                target[0][action] = reward
            
            # Иначе мы хотем увидеть максимальную награду, которую мы бы получили
            # если бы у нас была возможность совершить любое действие.
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
                
            self.model.fit(state, target, epochs=1, verbose=0)

    # Копируем веса в target_model.
    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)
        
    # Собираем всё воедино и обучаем модель.
    def train_model(self):
        
        games_number  = 1000
        goal_steps = 200
        
        steps = []
        for game_index in range(games_number):

            cur_state = env.reset().reshape(1,2)

            for step_index in range(goal_steps):
                
                # Выбираем действие.
                action = self.act(cur_state)
                new_state, reward, done, _ = env.step(action)

                # Запоминаем состояние.
                new_state = new_state.reshape(1,2)
                self.remember(cur_state, action, reward, new_state, done)

                # Обновляем веса обоих моделей
                self.replay()       
                self.target_train() 

                cur_state = new_state
                if done:
                    break
                    
                if (step_index%50==0):
                    print('Игра {}, итерация {}.'.format(game_index, step_index))
                    
            if step_index < 199:
                print("Выполнение завершено за {} игр.".format(game_index))
                self.save_model("DQNmodel.h5")
                break
                        
            print('Завершено {} игр из {}.'.format(game_index, games_number))

    # Сохраняем модель.
    def save_model(self, filepath):
        self.model.save(filepath)

In [271]:
env = gym.make("MountainCar-v0")
env.reset()

array([-0.43507919,  0.        ])

In [272]:
dqn_agent = DQN(env)

In [275]:
dqn_agent.train_model()

Итерация 0, игра 0.


KeyboardInterrupt: 

In [265]:
model_test(dqn_agent.model)

Средняя награда за 100 игр: -198.01
